In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("league_record.csv")

In [3]:
pd.set_option('display.max_columns', None)
df

,league_id,week,hour,minutes,HT,AT,HLS,ALS,HL2S,AL2S,HL3S,AL3S,HL4S,AL4S,HL5S,AL5S,hl_stat,al_stat,hl2_stat,al2_stat,hl3_stat,al3_stat,h_wins,a_wins,h_draws,a_draws,h_loss,a_loss,h_gf,a_gf,h_ga,a_ga,h_gd,a_gd,ht_points,at_points,ht_pos,at_pos
0,6095,1,12,52,EVE,MNC,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,2,2,1,-1,1,0,3,9,1
1,6095,1,12,52,BRN,TOT,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,1,0,-1,1,0,3,7,2
2,6095,1,12,52,BRI,FUL,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,1,1,-1,3,0,1,12
3,6095,1,12,52,LEI,WHU,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,2,1,1,2,1,-1,3,0,2,19
4,6095,1,12,52,WOL,ARS,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,1,6,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5315,6180,38,1,46,MNU,CRY,2,2,0,0,2,2,2,2,2,2,1,1,0,1,2,1,19,8,10,18,9,12,57,51,37,60,20,-9,67,42,4,15
5316,6180,38,1,46,WOL,EVE,2,1,0,3,0,1,0,1,2,0,2,0,0,2,0,0,11,14,9,7,18,17,37,42,45,52,-8,-10,42,49,16,11
5317,6180,38,1,46,BRN,MNC,0,5,1,2,0,2,2,2,1,0,0,2,0,2,0,0,8,25,10,7,20,6,37,74,67,30,-30,44,34,82,18,1
5318,6180,38,1,46,LEE,SOU,0,0,0,2,2,2,1,1,2,2,1,1,1,1,0,2,9,11,10,12,19,15,40,37,57,49,-17,-12,37,45,17,12


In [4]:
# Define function to check home and away scores and return result
def get_result(row):
    if row['HLS'] > row['ALS']:
        return 2  # home_win = 2
    elif row['HLS'] < row['ALS']:
        return 1  # away_win = 1
    else:
        return 0  # draw = 0

In [5]:
from sklearn.preprocessing import StandardScaler

df_result = df.copy()

# Apply function to each row of the dataframe to create a new column
df_result['result'] = df_result.apply(get_result, axis=1)

df_result = df_result[df_result['week'] > 4]  # remove rows with incomplete values

# remove irrelevant features
df_result = df_result.drop(["league_id", "week", "hour", "minutes", "HLS", "ALS", "HT", "AT", "hl_stat", "al_stat"], axis=1)  

df_result = df_result.reset_index(drop=True)  # reset the index values

# Scaling the dataset using standardization method
X = df_result.drop("result", axis=1)
y = df_result["result"]

# assume that X is your dataset with numerical features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# # concatenate the scaled numerical features with the categorical features
data_preprocessed = pd.concat([pd.DataFrame(X_scaled, columns=X.columns), y], axis=1)

data_preprocessed

,HL2S,AL2S,HL3S,AL3S,HL4S,AL4S,HL5S,AL5S,hl2_stat,al2_stat,hl3_stat,al3_stat,h_wins,a_wins,h_draws,a_draws,h_loss,a_loss,h_gf,a_gf,h_ga,a_ga,h_gd,a_gd,ht_points,at_points,ht_pos,at_pos,result
0,0.689649,-0.358320,0.489037,-1.114886,-0.235616,0.494614,-0.320331,0.629443,1.275768,-0.103095,1.089465,-1.087900,-1.145418,-1.516637,-1.430927,-0.535693,-1.190896,-1.402492,-1.483105,-1.535425,-1.378625,-1.252640,-0.151777,-0.343698,-1.334109,-1.496887,-0.695553,0.494204,0
1,-1.110094,1.286426,2.983123,-1.114886,-1.121502,-1.170422,0.516539,0.629443,-1.069339,1.068106,1.089465,-1.087900,-0.951016,-1.322848,-1.730611,-1.434179,-1.190896,-0.995092,-1.289106,-1.470815,-1.643764,-1.385188,0.345588,-0.131707,-1.215262,-1.496887,-1.041911,0.839587,2
2,-0.210222,-0.358320,-0.342326,-1.114886,-1.121502,-0.337904,1.353409,0.629443,-1.069339,-1.274296,-0.081421,-1.087900,-1.339820,-1.129060,-1.131243,-1.434179,-1.190896,-1.198792,-1.483105,-1.535425,-1.312340,-1.650286,-0.222829,0.080283,-1.452956,-1.319162,0.343520,-0.196564,0
3,-1.110094,-0.358320,-1.173688,2.441393,-1.121502,1.327131,-1.157201,1.513003,-1.069339,1.068106,-0.081421,1.250507,-1.534222,-0.547696,-1.131243,-1.733674,-0.986737,-1.606192,-1.806438,-0.889330,-1.444910,-1.650286,-0.435986,0.786919,-1.631227,-0.845228,1.555772,-1.578099,1
4,0.689649,0.464053,2.151761,-0.225816,0.650270,0.494614,0.516539,-0.254116,1.275768,1.068106,1.089465,-1.087900,-0.951016,-0.935272,-1.730611,-1.733674,-1.190896,-1.198792,-1.159772,-1.341596,-1.378625,-1.517737,0.203484,0.150947,-1.215262,-1.200678,-1.215090,-0.887332,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4755,-1.110094,-1.180692,0.489037,0.663254,0.650270,0.494614,0.516539,0.629443,-1.069339,-0.103095,1.089465,0.081304,2.159411,0.033669,1.266227,3.657237,0.238214,0.838209,1.814888,1.436610,0.543632,2.061075,1.411372,-0.626352,2.231305,0.754299,-1.041911,0.839587,0
4756,-1.110094,1.286426,-1.173688,-0.225816,-1.121502,-0.337904,0.516539,-1.137676,-1.069339,1.068106,-1.252307,-1.087900,0.604197,1.196398,0.966543,0.362792,2.075640,1.856709,0.521557,0.855125,1.073910,1.530880,-0.578090,-0.697016,0.745716,1.168991,1.036235,0.148820,2
4757,-0.210222,0.464053,-1.173688,0.663254,0.650270,0.494614,-0.320331,-1.137676,-1.069339,1.068106,-1.252307,-1.087900,0.020992,3.328067,1.266227,0.362792,2.483957,-0.383992,0.521557,2.922628,2.532174,0.072846,-2.141239,3.118817,0.270327,3.123968,1.382593,-1.578099,1
4758,-1.110094,0.464053,0.489037,0.663254,-0.235616,-0.337904,0.516539,0.629443,0.103214,-0.103095,-1.252307,1.250507,0.215394,0.615033,1.266227,1.860267,2.279799,1.449309,0.715557,0.532078,1.869327,1.332058,-1.217560,-0.838343,0.448598,0.932024,1.209414,0.321512,0


In [6]:
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Load your dataset into a pandas DataFrame
dataset = data_preprocessed.copy()

# Specify the target variable
target_var = 'result'

# Specify the number of folds for k-fold cross-validation
n_splits = 80

# Initialize the KFold cross-validation object
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)

# Initialize a list to store the accuracy scores for each fold
accuracy_scores = []

# Loop over each fold
for train_idx, test_idx in kfold.split(dataset):
    # Split the data into training and testing sets for the current fold
    X_train, X_test = dataset.drop(target_var, axis=1).iloc[train_idx], dataset.drop(target_var, axis=1).iloc[test_idx]
    y_train, y_test = dataset[target_var].iloc[train_idx], dataset[target_var].iloc[test_idx]
    
    # Initialize a random forest classifier with default hyperparameters
    rf = RandomForestClassifier(random_state=42)
    
    # Fit the classifier to the training data
    rf.fit(X_train, y_train)
    
    # Make predictions on the test data
    y_pred = rf.predict(X_test)
    
    # Calculate the accuracy score for the current fold
    accuracy = accuracy_score(y_test, y_pred)
    
    # Append the accuracy score to the list
    accuracy_scores.append(accuracy)

# Calculate the mean accuracy score across all folds
mean_accuracy = sum(accuracy_scores) / n_splits

print(f"Mean accuracy score: {mean_accuracy}")

Mean accuracy score: 0.6340148305084742


In [7]:
# 4420 Observations
# Mean accuracy score: 0.6326167076167074 - Classifier(Scaled) - 40 splits
# Mean accuracy score: 0.635329417773238- Classifier(Scaled) - 50 splits
# MMean accuracy score: 0.6348327779834632 - Classifier(Scaled) - 60 splits
# Mean accuracy score: 0.6316928854875286 - Classifier(Scaled) - 70 splits
# Mean accuracy score: 0.6337337662337659 - Classifier(Scaled) - 80 splits

In [8]:
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

# Load your dataset into a pandas DataFrame
dataset = data_preprocessed.copy()
# Specify the target variable
target_var = 'result'

# Specify the number of folds for k-fold cross-validation
n_splits = 50

# Initialize the KFold cross-validation object
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)

# Initialize a list to store the R-squared scores for each fold
r2_scores = []

# Loop over each fold
for train_idx, test_idx in kfold.split(dataset):
    # Split the data into training and testing sets for the current fold
    X_train, X_test = dataset.drop(target_var, axis=1).iloc[train_idx], dataset.drop(target_var, axis=1).iloc[test_idx]
    y_train, y_test = dataset[target_var].iloc[train_idx], dataset[target_var].iloc[test_idx]
    
    # Initialize a random forest regressor with default hyperparameters
    rf = RandomForestRegressor(random_state=42)
    
    # Fit the regressor to the training data
    rf.fit(X_train, y_train)
    
    # Make predictions on the test data
    y_pred = rf.predict(X_test)
    
    # Calculate the R-squared for the current fold
    r2 = r2_score(y_test, y_pred)
    
    # Append the R-squared to the list
    r2_scores.append(r2)

# Calculate the mean R-squared across all folds
mean_r2 = sum(r2_scores) / n_splits

print(f"Mean R-squared: {mean_r2}")

Mean R-squared: 0.24326457253332198


In [9]:
# 4420 Observations
# Mean R-squared: 0.24548274665914768 - 40 splits
# Mean R-squared: 0.24930608614772112 - Regressor(Scaled) - 50 splits ------->
# Mean R-squared: 0.24661057530469332 - Regressor(Scaled) - 70 splits

In [11]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import time

driver = webdriver.Chrome("chrome_driver/chromedriver.exe")

# Navigate to a web page
driver.get("https://vsmobile.bet9ja.com/mobile/themes/?sk=bet9ja&t=b61c29e6-9348-4c58-af90-378760a74693&game"
               "=league_premier&pid=14001,14003,14011,14012,14014,14015,14016,"
               "14017&v=0&text=Premier&lang=en_GB#resutls&ui_state=dialog")
driver.maximize_window()

print("waiting ...")
time.sleep(3)


driver.find_element(By.XPATH, "//div[@class='ui-panel-wrapper']//i[@class='fa fa-bars icon-menu']").click()
time.sleep(1)

driver.find_element(By.XPATH, "//a[@id='a_bet_results']").click()
time.sleep(3)

driver.find_element(By.XPATH, "//div[@id='results_content']").click()
time.sleep(1)

driver.send_keys(Keys.CONTROL + 'a')
driver.send_keys(Keys.CONTROL + 'c')

# Click on the element
# element.click()
# Get the copied text from the clipboard
import pyperclip
copied_text = pyperclip.paste()
 
# # Print the copied text
# print(copied_text)
time.sleep(3)
# Close the browser session
driver.quit()

C:\Users\CJay\AppData\Local\Temp\ipykernel_10808\2718491913.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("chrome_driver/chromedriver.exe")


waiting ...
